In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import layers, Model
from keras import ops
from functools import partial

@tf.function
def con(x, y):
    
  return tf.stack([x, y], axis=2)

@tf.function
def msloss(ytrue, ypred):
  """
  modified loss function with rescaling

  needs some work vor vae
  """
  #ytrue[:] = ytrue[:,tf.where(tf.sum(ytrue[:,:]))]
  #ypred[:] = ypred[:,tf.where(tf.sum(ytrue[:,:]))]
  #ytrue = layers.Input(shape=(19,3))(ytrue)
  #ypred = layers.input(shape=(19,3))

  yte = tf.tanh(ytrue[:,:,1])
  ytp = tf.tanh(ytrue[:,:,2])

  ype = tf.tanh(ypred[:,:,1])
  ypp = tf.tanh(ypred[:,:,2])

  #ytr = ypred[:,:,0]/
  #ypr
    
  yt = tf.concat([ytrue[:,:,0], yte,ytp], axis=1)
  yp = tf.concat([ypred[:,:,0], ype,ypp], axis=1)

  return tf.reduce_mean(tf.reduce_sum(tf.square(yt-yp)))

@tf.function
def vmsloss(ytrue, ypred):
  """
  modified loss function with rescaling

  needs some work vor vae
  """
  #ytrue[:] = ytrue[:,tf.where(tf.sum(ytrue[:,:]))]
  #ypred[:] = ypred[:,tf.where(tf.sum(ytrue[:,:]))]
  #ytrue = layers.Input(shape=(19,3))(ytrue)
  #ypred = layers.input(shape=(19,3))

  yte = tf.tanh(ytrue[1])
  ytp = tf.tanh(ytrue[2])

  ype = tf.tanh(ypred[1])
  ypp = tf.tanh(ypred[2])

  yt = tf.concat([ytrue[0], yte,ytp], axis=1)
  yp = tf.concat([ypred[0], ype,ypp], axis=1)
  if yp.shape != yt.shape:
    yp = tf.reshape(yp, yt.shape)

  return tf.reduce_mean(tf.reduce_sum(tf.square(yt-yp)))

@tf.function
def tmp(inputs, see):
    """For VAE implementation a lot of code is taken from
    https://keras.io/examples/generative/vae/
    
    import numpy as np
    import tensorflow as tf
    import keras
    from keras import ops
    from keras import layers
    class Sampling(layers.Layer):
      Uses (z_mean, z_log_var) to sample z, the vector encoding a digit.

      def __init__(self, **kwargs):
          super().__init__(**kwargs)
          self.seed_generator = keras.random.SeedGenerator(1337)

      def call(self, inputs):"""
    batch = ops.shape(inputs)[0]
    dim = ops.shape(inputs)[1]
    epsilon = keras.random.normal(shape=(batch, dim), seed=see)
    return ops.exp(0.5 * inputs) * epsilon

    #return Sampling

#Sampling = tmp()

def dnencoder(vae=0, latent_dim=3):
  encoder_inputs = layers.Input(shape=(19,3,))
  x = layers.BatchNormalization()(encoder_inputs)
  x = layers.Flatten()(x)
  x = layers.Dense(32)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(16)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)

  if vae:
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    #tf.print(z_mean[0])
    encoder = Model(encoder_inputs, [z_mean, z_log_var], name="encoder")

  else:
    x = layers.Dense(latent_dim)(x)
    encoder = Model(encoder_inputs, x, name="encoder")

  return encoder

def dndecoder(vae=1, latent_dim=3):

  decoder_inputs = layers.Input(shape=(3,)) #keras doesn want any other shape than 3
  x = layers.Dense(16)(decoder_inputs)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(32)(x)
  x = layers.BatchNormalization()(x)
  x = layers.LeakyReLU()(x)
  x = layers.Dense(19*3)(x)
  x = layers.BatchNormalization()(x)
  decoder_outputs = layers.Reshape((19,3))(x)
  return Model(decoder_inputs, decoder_outputs, name="decoder")


def cnencoder(vae=0, latent_dim=8):
  encoder_inputs = layers.Input(shape=(19,3,1))
  x = layers.ZeroPadding2D(padding=(1,0))(encoder_inputs)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(16, kernel_size=(3, 3), strides=1, padding="valid")(x)
  x = layers.ReLU()(x)
  x = layers.AveragePooling2D((3, 1))(x)
  x = layers.Conv2D(32, kernel_size=(3, 1), strides=1, padding="valid")(x)
  x = layers.ReLU()(x)
  x = layers.AveragePooling2D((3, 1))(x)
  x = layers.Flatten()(x)
  if vae:
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    encoder = Model(encoder_inputs, [z_mean, z_log_var], name="encoder")
  else:
    x = layers.Dense(latent_dim)(x)
    encoder = Model(encoder_inputs, x, name="encoder")
  return encoder

def cndecoder(vae=0, latent_dim=8):

  decoder_inputs = layers.Input(shape=(latent_dim,))
  #x = layers.BatchNormalization()
  x = layers.Dense(64)(decoder_inputs)
  x = layers.ReLU()(x)
  x = layers.Reshape((2,1,32))(x)
  x = layers.Conv2D(32,(3, 1), strides=1, padding='same')(x)
  x = layers.ReLU()(x)
  x = layers.UpSampling2D((3, 1))(x)
  x = layers.ZeroPadding2D(((0,0),(1,1)))(x)
  x = layers.Conv2D(16, (3, 1), strides=1, padding='same')(x)
  x = layers.ReLU()(x)
  x = layers.UpSampling2D((3, 1))(x)
  x = layers.ZeroPadding2D(((1,0),(0,0)))(x)
  decoder_outputs = layers.Conv2D(1, (3, 3), strides=1, padding='same')(x)
  return Model(decoder_inputs, decoder_outputs, name="decoder")

2025-03-28 20:03:31.120137: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 20:03:31.194613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-28 20:03:33.930386: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/Software/users/modules/9/software/anaconda3/2023.03/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.11.909) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn

In [2]:
@tf.keras.utils.register_keras_serializable()
class AE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def call(self, inputs):
        z= self.encoder(inputs)
        return self.decoder(z)

beta = .01
@tf.keras.utils.register_keras_serializable()
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.seed=keras.random.SeedGenerator(1337)
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var = self.encoder(data[0])
            z_mean = z_mean + tmp(z_log_var, self.seed)
            reconstruction = self.decoder(z_mean)
            reconstruction_loss = vmsloss(data[0], reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            
            total_loss = (1-beta)*reconstruction_loss + beta*kl_loss
        
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var = self.encoder(data[0])
        reconstruction = self.decoder(z_mean)
        reconstruction_loss = vmsloss(data[1], reconstruction)

        # ✅ KL divergence
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.cast(tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1)), tf.float32)

        # ✅ Total loss with β weighting
        total_loss = (1.0 - beta) * reconstruction_loss + beta * kl_loss

        # ✅ Update metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var = self.encoder(inputs)
        #print(z_mean)
        reconstruction = self.decoder(z_mean)
        return reconstruction
    
    

In [3]:
import h5py
from sklearn.model_selection import train_test_split
from functools import partial

dset = h5py.File('./content/background_for_training2.h5', 'r')
#dset = {key: dset[key][()] for key in dset.keys()}
"""
Contains keys:
    Particles_Classes : 4 classes of Particles
    Particles_Names : Names of the Particles
    Particles : The data (n, 19,4)
        19 : Indexes are
            - 0 : Missing Transverse Energy
            - 1:4 Up to 4 electrons
            - 4:8 Up to 4 muons
            - 8-18 Up to 10 jets
        Subdimension 4 by idx:
            - 0 : pT (transverse momentum)
            - 1 : eta (pseudorapidity)
            - 2 : phi (azimuthal angle)
            - 3 : class (0=Nothing, 1=Met,2=electron,3=muon,4=jet)
And when something doesnt make sense (ie [:,0,1:4]) its just zero
"""
data = (dset['Particles'])[:,:,:3]
del dset
# Do z score norm to aid in training : They dont specify how they made O(1) : And I assume they mean across all defined objects
#detected_bmap = (data[:,:,3] != 0) # Select defined entries
#mean_pt = tf.reduce_mean(data[detected_bmap, 0])
#std_pt = tf.math.reduce_std(data[detected_bmap, 0])
#data[:,:,0] = ((data[:,:,0] - mean_pt) / std_pt)

sloss = tf.function(partial(msloss))

dn_ae = AE(dnencoder(), dndecoder())
dn_ae.compile(optimizer="Adam", loss=sloss)

cn_ae = AE(cnencoder(), cndecoder())
cn_ae.compile(optimizer=keras.optimizers.Adam(), loss=sloss)

dn_vae = VAE(dnencoder(vae=1), dndecoder(vae=1))
dn_vae.compile(optimizer="Adam", loss=sloss)

cn_vae = VAE(cnencoder(vae=1), cndecoder(vae=1))
cn_vae.compile(optimizer=keras.optimizers.Adam(), loss=sloss)



In [4]:
train, test = train_test_split(data, test_size=0.5)
test, val = train_test_split(test, test_size=0.8)

In [5]:
dn_ae.fit(train, train, validation_data=(test, test), epochs=100, batch_size=1024)
dn_ae.save_weights('edn_ae_weights.weights.h5')
#dn_ae.save('dn_ae_model.keras')

In [6]:
#con([1,2,3],[4,5,6])
#dn_vae.compile(optimizer='Adam')

In [7]:
#dn_vae = VAE(dnencoder(vae=1), dndecoder(vae=1))
#dn_vae.compile(optimizer="Adam", loss=sloss)

#dn_vae.predict(train[:5])

In [8]:
dn_vae = VAE(dnencoder(vae=1), dndecoder(vae=1))
dn_vae.compile(optimizer="Adam", loss=sloss)

dn_vae.fit(train, train, validation_data=(test, test), epochs=100, batch_size=1024)
dn_vae.save_weights('edn_vae_weights.weights.h5')
#dn_vae.save('dn_vae_model.keras')

In [9]:
cn_vae = VAE(cnencoder(vae=1), cndecoder(vae=1))
cn_vae.compile(optimizer="Adam", loss=sloss)

cn_vae.fit(train, train, validation_data=(test, test), epochs=100, batch_size=1024)
cn_vae.save_weights('ecn_vae_weights.weights.h5')

cn_ae.fit(train, train, validation_data=(test, test), epochs=100, batch_size=1024)
cn_ae.save_weights('ecn_ae_weights.weights.h5')
#cn_vae.save('cn_vae_model.keras')

#cn_vae = VAE(cnencoder(vae=1), cndecoder())
#cn_vae.compile(optimizer=keras.optimizers.Adam(), loss=sloss)

## my plotting didn't work so using felix's for now

In [ ]:
import numpy as np

# Gotta Love Chat GPT I wrote none of this 
# ================================
# ANOMALY SCORE COMPUTATION 
# ================================

# === 1. IO-Based Anomaly Score ===
def compute_io_score(y_true, y_pred):
    """
    Compute input-output (IO) anomaly score based on reconstruction loss (MSE).
    Used for: AE and VAE models
    
    Args:
        y_true (np.array): True input data [batch_size, height, width, channels]
        y_pred (np.array): Reconstructed output from the model [batch_size, height, width, channels]
    
    Returns:
        np.array: Anomaly score for each sample based on MSE
    """
    return np.square(y_true - y_pred)

# === 2. KL-Based Anomaly Score ===
def compute_kl_score(z_mean, z_log_var):
    """
    Compute KL-based anomaly score based on latent space properties.
    Used for: VAE models
    
    Args:
        z_mean (np.array): Mean of the latent space [batch_size, latent_dim]
        z_log_var (np.array): Log variance of the latent space [batch_size, latent_dim]
    
    Returns:
        np.array: KL divergence score for each sample
    """
    kl_div = -0.5 * np.sum(1 + z_log_var - np.square(z_mean) - np.exp(z_log_var), axis=1)
    # Replace inf 
    kl_div = np.nan_to_num(kl_div, nan=0.0, posinf=1e20, neginf=-1e20)
    return kl_div

# === 3. Rz-Based Anomaly Score ===
def compute_rz_score(z_mean, z_log_var):
    """
    Compute Rz-based anomaly score (Euclidean distance from origin in latent space).
    Used for: VAE models
    
    Args:
        z_mean (np.array): Mean of the latent space [batch_size, latent_dim]
        z_log_var (np.array): Log variance of the latent space [batch_size, latent_dim]
    
    Returns:
        np.array: Rz score for each sample
    """
    rz = np.sqrt(np.sum(np.square(z_mean), axis=-1) + np.sum(np.exp(z_log_var), axis=-1))
    rz = np.nan_to_num(rz, nan=0.0, posinf=1e20, neginf=-1e20)
    return rz

In [10]:
path= './content/'

dn_vae = VAE(dnencoder(vae=1), dndecoder(vae=1))
dn_vae.compile(optimizer="Adam", loss=sloss)
#dn_vae.fit(train[:2], train[:2], batch_size=1)
dn_vae.load_weights('./rdn_vae_weights.weights.h5')

cn_vae = VAE(cnencoder(vae=1), cndecoder(vae=1))
cn_vae.compile(optimizer="Adam", loss=sloss)
#cn_vae.fit(train[:2], train[:2], batch_size=1)
cn_vae.load_weights('./rcn_vae_weights.weights.h5')

/Software/users/modules/9/software/anaconda3/2023.03/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/Software/users/modules/9/software/anaconda3/2023.03/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import h5py

#dn_ae = AE(dnencoder(), dndecoder())
#dn_ae.compile(optimizer="Adam", loss=sloss)
#dn_ae.fit(train[:2], train[:2], batch_size=1)
#dn_ae.load_weights('./dn_ae_weights.weights.h5')

#cn_ae = AE(cnencoder(), cndecoder())
#cn_ae.compile(optimizer="Adam", loss=sloss)
#cn_ae.fit(train[:2], train[:2], batch_size=1)
#cn_ae.load_weights('./content/cn_ae_weights.weights.h5')

# For ROC we also need a background set lets use 1e5 background events
background = h5py.File('./content/background_for_training2.h5', 'r')['Particles'][:,:,:3][:1000000,:,:3]
# ================================
# PLOT OUTLINE BASED ON THE PAPER
# ================================
def compute_io_score(y_true, y_pred):
    """
    Compute input-output (IO) anomaly score based on reconstruction loss (MSE).
    Used for: AE and VAE models

    Args:
        y_true (np.array): True input data [batch_size, height, width, channels]
        y_pred (np.array): Reconstructed output from the model [batch_size, height, width, channels]

    Returns:
        np.array: Anomaly score for each sample based on MSE
    """
    y_pred = tf.reshape(y_pred, y_true.shape)
    return np.square(y_true - y_pred)
# === FIGURE II: ROC Curves for CNN and DNN Models ===
# Data:
# - x-axis → False Positive Rate (FPR)
# - y-axis → True Positive Rate (TPR)
# - Models:
#   - CNN VAE (using IO, KL divergence, Rz scores)
#   - DNN VAE (using IO, KL divergence, Rz scores)
#   - CNN AE (using IO score)
#   - DNN AE (using IO score)
# Benchmark Models:
# - LQ → bτ
# - A → 4ℓ
# Method:
# - Compute ROC curves based on anomaly scores:
#   - MSE for AEs
#   - KL and Rz for VAEs
#   - Plot separate ROC curves for CNN and DNN
def figureII():
    def do_roc(ax, score, y_true, text):
        fpr, tpr, _ = roc_curve(y_true, score)
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr, label=f'{text} (AUC = {roc_auc:.3f})')
    fig, ax = plt.subplots(2, 2, figsize=(16, 16))
    ax = ax.flatten()

    # CNN LQ -> bτ
    #Load SIGNAL ONLY Data
    dset = h5py.File('./content/leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    #Concatenate dset and background - Load as much bkground as data
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    #dset[:,:,0] = ((dset[:,:,0] - mean_pt) / std_pt)

  #  CNN_VAE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_vae_beta1.0.keras')
  #  CNN_AE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_ae.keras')

   # cnn_io_vae = compute_io_score(dset, CNN_VAE.predict(dset)[:,:,:,0])
    #do_roc(ax[0], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

   # cnn_dkl_vae = compute_kl_score(*CNN_VAE.encoder.predict(dset)[0:2])
    #do_roc(ax[0], cnn_dkl_vae, labels, "VAE D_{KL}")

   # cnn_Rz_vae = compute_rz_score(*CNN_VAE.encoder.predict(dset)[0:2])
    #do_roc(ax[0], cnn_Rz_vae, labels, "VAE R_Z")

   # cnn_io_ae = compute_io_score(dset, CNN_AE.predict(dset)[:,:,:,0])
    #do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "leptoquark")#"IO AE")

    cnn_io_ae = compute_io_score(dset, cn_ae.predict(dset))
    do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "leptoquark")#"IO AE")
    
    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "leptoquark")#"IO AE")

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset))
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "leptoquark")#"IO AE")

    dset = h5py.File('./content/hChToTauNu_13TeV_PU20_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    #Concatenate dset and background - Load as much bkground as data
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "hChToTauNu")#"IO AE")

    cnn_io_ae = compute_io_score(dset, cn_ae.predict(dset))
    do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "hChToTauNu")
    
    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "hChToTauNu")#"IO AE")

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset))
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "hChToTauNu")#"IO AE")

    dset = h5py.File('./content/Ato4l_lepFilter_13TeV_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])
    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "Ato4l")#"IO AE")

    cnn_io_ae = compute_io_score(dset, cn_ae.predict(dset))
    do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "Ato4l")
    
    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "Ato4l")#"IO AE")

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset))
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "Ato4l")#"IO AE")

    dset = h5py.File('./content/hToTauTau_13TeV_PU20_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "hToTauTau")

    cnn_io_ae = compute_io_score(dset, cn_ae.predict(dset))
    do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "hToTauTau")
    
    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "hToTauTau")#"IO AE")

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset))
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "hToTauTau")#"IO AE")

    # DNN LQ -> bτ
    #DNN_VAE = tf.keras.models.load_model('/Code/Replicate/Models/full_dnn_vae_beta1.0.keras')
    #DNN_AE = tf.keras.models.load_model('/Code/Replicate/Models/full_dnn_ae.keras')
    #dnn_ae.load_weights('/content/dnn.weights.h5')


#    dnn_io_vae = compute_io_score(dset, DNN_VAE.predict(dset))
 #   do_roc(ax[1], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

  #  dnn_dkl_vae = compute_kl_score(*DNN_VAE.encoder.predict(dset)[0:2])
   # do_roc(ax[1], dnn_dkl_vae, labels, "VAE D_{KL}")

   # dnn_Rz_vae = compute_rz_score(*DNN_VAE.encoder.predict(dset)[0:2])
    #do_roc(ax[1], dnn_Rz_vae, labels, "VAE R_Z")

    #dnn_io_ae = msloss(dset, dn_ae.predict(dset))
    #do_roc(ax[1], tf.reduce_mean(dnn_io_ae), labels, "IO AE")

    """
    # CNN A-> 4l
    dset = h5py.File('Ato4l_lepFilter_13TeV_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    # Concatenate dset and background
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    cnn_io_vae = compute_io_score(dset, CNN_VAE.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    cnn_dkl_vae = compute_kl_score(*CNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_dkl_vae, labels, "VAE D_{KL}")

    cnn_Rz_vae = compute_rz_score(*CNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_Rz_vae, labels, "VAE R_Z")

    cnn_io_ae = compute_io_score(dset, CNN_AE.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # DNN A-> 4l
    dnn_io_vae = compute_io_score(dset, DNN_VAE.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    dnn_dkl_vae = compute_kl_score(*DNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_dkl_vae, labels, "VAE D_{KL}")

    dnn_Rz_vae = compute_rz_score(*DNN_VAE.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_Rz_vae, labels, "VAE R_Z")

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[3], dnn_io_ae.mean(axis=(1,2)), labels, "IO AE")
"""


    # And formating
    legend_text = [
        "CNN ROC LQ → bτ",
        "DNN ROC LQ → bτ",
        "CNN ROC A → 4ℓ",
        "DNN ROC A → 4ℓ"
    ]
    for i, a in enumerate(ax):
        a.set_xlim(1e-6, 1)
        a.set_ylim(1e-6, 1)
        a.set_xscale('log')
        a.set_yscale('log')
        a.set_xlabel('False Positive Rate')
        a.set_ylabel('True Positive Rate')
        a.set_xticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.set_yticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.grid(False)
        a.plot([1e-6, 1], [1e-6, 1], color='grey', linestyle='--', alpha=0.4)
        a.axvline(x=1e-5, color='red', linestyle='--', label='Label')
        # Add Legend Handle
        handles, labels = a.get_legend_handles_labels()
        # Create the custom text as a Line2D object
        #extra = Line2D([0], [0], color='none', label='')
        # Insert at the beginning of the list
        #handles.insert(0, extra)
        #labels.insert(0, legend_text[i])
        a.legend(handles=handles, labels=labels, loc='lower right')
    plt.savefig('figur1')
    plt.show()

figureII()



2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 2921.0601
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 2678.2769
21284/21284 ━━━━━━━━━━━━━━━━━━━━ 19s 882us/step
21284/21284 ━━━━━━━━━━━━━━━━━━━━ 46s 2ms/step
21284/21284 ━━━━━━━━━━━━━━━━━━━━ 18s 828us/step
21284/21284 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step
47517/47517 ━━━━━━━━━━━━━━━━━━━━ 42s 892us/step
47517/47517 ━━━━━━━━━━━━━━━━━━━━ 72s 2ms/step


In [ ]:
#error

In [ ]:


import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

from matplotlib.lines import Line2D
# For ROC we also need a background set lets use 1e5 background events
background = h5py.File('{}background_for_training2.h5'.format(path), 'r')['Particles'][:1000000,:,:3]
# ================================
# PLOT OUTLINE BASED ON THE PAPER
# ================================

# === FIGURE II: ROC Curves for CNN and DNN Models ===
# Data:
# - x-axis → False Positive Rate (FPR)
# - y-axis → True Positive Rate (TPR)
# - Models:
#   - CNN VAE (using IO, KL divergence, Rz scores)
#   - DNN VAE (using IO, KL divergence, Rz scores)
#   - CNN AE (using IO score)
#   - DNN AE (using IO score)
# Benchmark Models:
# - LQ → bτ
# - A → 4ℓ
# Method:
# - Compute ROC curves based on anomaly scores:
#   - MSE for AEs
#   - KL and Rz for VAEs
#   - Plot separate ROC curves for CNN and DNN
def figureII():
    def do_roc(ax, score, y_true, text):
        try:
            fpr, tpr, _ = roc_curve(y_true, score)
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, label=f'{text} (AUC = {roc_auc:.3f})')
        except:
            pass
    fig, ax = plt.subplots(2, 2, figsize=(16, 16))
    ax = ax.flatten()

    # CNN LQ -> bτ
    # Load SIGNAL ONLY Data
    dset = h5py.File('{}/leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5'.format(path), 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    # Concatenate dset and background - Load as much bkground as data
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    #dset[:,:,0] = ((dset[:,:,0] - mean_pt) / std_pt)

    #CNN_VAE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_vae_beta0.01.keras')
    #CNN_AE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_ae.keras')

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset)[:,:,:,0])
    do_roc(ax[0], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    cnn_dkl_vae = compute_kl_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[0], cnn_dkl_vae, labels, "VAE D_{KL}")

    cnn_Rz_vae = compute_rz_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[0], cnn_Rz_vae, labels, "VAE R_Z")

    cnn_io_ae = compute_io_score(dset, cn_vae.predict(dset)[:,:,:,0])
    do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")



    # DNN LQ -> bτ
    #DNN_VAE = tf.keras.models.load_model('content/full_dnn_vae_beta0.01.keras')
    #DNN_AE = tf.keras.models.load_model('content/full_dnn_ae.keras')

    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[1], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    dnn_dkl_vae = compute_kl_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[1], dnn_dkl_vae, labels, "VAE D_{KL}")

    dnn_Rz_vae = compute_rz_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[1], dnn_Rz_vae, labels, "VAE R_Z")

    dnn_io_ae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # CNN A-> 4l
    dset = h5py.File('{}Ato4l_lepFilter_13TeV_filtered.h5'.format(path), 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    # Concatenate dset and background
    n_points = dset.shape[0]
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    cnn_dkl_vae = compute_kl_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_dkl_vae, labels, "VAE D_{KL}")

    cnn_Rz_vae = compute_rz_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_Rz_vae, labels, "VAE R_Z")

    cnn_io_ae = compute_io_score(dset, cn_ae.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # DNN A-> 4l
    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    dnn_dkl_vae = compute_kl_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_dkl_vae, labels, "VAE D_{KL}")

    dnn_Rz_vae = compute_rz_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_Rz_vae, labels, "VAE R_Z")

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[3], dnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # And formating
    legend_text = [
        "CNN ROC LQ → bτ",
        "DNN ROC LQ → bτ",
        "CNN ROC A → 4ℓ",
        "DNN ROC A → 4ℓ"
    ]
    for i, a in enumerate(ax):
        a.set_xlim(1e-6, 1)
        a.set_ylim(1e-6, 1)
        a.set_xscale('log')
        a.set_yscale('log')
        a.set_xlabel('False Positive Rate')
        a.set_ylabel('True Positive Rate')
        a.set_xticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.set_yticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.grid(False)
        a.plot([1e-6, 1], [1e-6, 1], color='grey', linestyle='--', alpha=0.4)
        a.axvline(x=1e-5, color='red', linestyle='--', label='Label')
        # Add Legend Handle
        handles, labels = a.get_legend_handles_labels()
        # Create the custom text as a Line2D object
        extra = Line2D([0], [0], color='none', label='')
        # Insert at the beginning of the list
        handles.insert(0, extra)
        labels.insert(0, legend_text[i])
        a.legend(handles=handles, labels=labels, loc='lower right')
    plt.savefig('figur2')
    plt.show()

figureII()


# === FIGURE III: Additional ROC Curves for CNN and DNN Models ===
# Data:
# - x-axis → False Positive Rate (FPR)
# - y-axis → True Positive Rate (TPR)
# - Models:
#   - CNN VAE (using IO, KL divergence, Rz scores)
#   - DNN VAE (using IO, KL divergence, Rz scores)
#   - CNN AE (using IO score)
#   - DNN AE (using IO score)
# Benchmark Models:
# - h± → τν
# - h0 → ττ
# Method:
# - Same as Figure II, but using different benchmark scenarios

def figureIII():
    def do_roc(ax, score, y_true, text):
        try:
            fpr, tpr, _ = roc_curve(y_true, score)
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, label=f'{text} (AUC = {roc_auc:.3f})')
        except:
            pass
    fig, ax = plt.subplots(2, 2, figsize=(16, 16))
    ax = ax.flatten()

    # CNN LQ -> bτ
    # Load SIGNAL ONLY Data
    dset = h5py.File('./content/hChToTauNu_13TeV_PU20_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    # Concatenate dset and background - Load as much bkground as data
    n_points = np.minimum(dset.shape[0], background.shape[0])
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    #dset[:,:,0] = ((dset[:,:,0] - mean_pt) / std_pt)

    #CNN_VAE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_vae_beta0.01.keras')
    #CNN_AE = tf.keras.models.load_model('/Code/Replicate/Models/full_cnn_ae.keras')

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset)[:,:,:,0])
    do_roc(ax[0], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    cnn_dkl_vae = compute_kl_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[0], cnn_dkl_vae, labels, "VAE D_{KL}")

    cnn_Rz_vae = compute_rz_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[0], cnn_Rz_vae, labels, "VAE R_Z")

    cnn_io_ae = compute_io_score(dset, cn_ae.predict(dset)[:,:,:,0])
    do_roc(ax[0], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")



    # DNN LQ -> bτ
    #DNN_VAE = tf.keras.models.load_model('/Code/Replicate/Models/full_dnn_vae_beta0.01.keras')
    #DNN_AE = tf.keras.models.load_model('/Code/Replicate/Models/full_dnn_ae.keras')

    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[1], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    dnn_dkl_vae = compute_kl_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[1], dnn_dkl_vae, labels, "VAE D_{KL}")

    dnn_Rz_vae = compute_rz_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[1], dnn_Rz_vae, labels, "VAE R_Z")

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[1], dnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # CNN A-> 4l
    dset = h5py.File('./content/hToTauTau_13TeV_PU20_filtered.h5', 'r')
    dset = dset['Particles'][:,:,:3]
    labels = np.ones(dset.shape[0])

    # Concatenate dset and background
    n_points = np.minimum(dset.shape[0], background.shape[0])
    dset = np.concatenate([dset, background[:n_points]], axis=0)
    labels = np.concatenate([labels, np.zeros(n_points)])

    cnn_io_vae = compute_io_score(dset, cn_vae.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    cnn_dkl_vae = compute_kl_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_dkl_vae, labels, "VAE D_{KL}")

    cnn_Rz_vae = compute_rz_score(*cn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[2], cnn_Rz_vae, labels, "VAE R_Z")

    cnn_io_ae = compute_io_score(dset, cn_vae.predict(dset)[:,:,:,0])
    do_roc(ax[2], cnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # DNN A-> 4l
    dnn_io_vae = compute_io_score(dset, dn_vae.predict(dset))
    do_roc(ax[3], dnn_io_vae.mean(axis=(1,2)), labels, "IO VAE")

    dnn_dkl_vae = compute_kl_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_dkl_vae, labels, "VAE D_{KL}")

    dnn_Rz_vae = compute_rz_score(*dn_vae.encoder.predict(dset)[0:2])
    do_roc(ax[3], dnn_Rz_vae, labels, "VAE R_Z")

    dnn_io_ae = compute_io_score(dset, dn_ae.predict(dset))
    do_roc(ax[3], dnn_io_ae.mean(axis=(1,2)), labels, "IO AE")


    # And formating
    legend_text = [
        "CNN ROC h± → τν",
        "DNN ROC h± → τν",
        "CNN ROC h0 → ττ",
        "DNN ROC h0 → ττ"
    ]
    for i, a in enumerate(ax):
        a.set_xlim(1e-6, 1)
        a.set_ylim(1e-6, 1)
        a.set_xscale('log')
        a.set_yscale('log')
        a.set_xlabel('False Positive Rate')
        a.set_ylabel('True Positive Rate')
        a.set_xticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.set_yticks([10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0])
        a.grid(False)
        a.plot([1e-6, 1], [1e-6, 1], color='grey', linestyle='--', alpha=0.4)
        a.axvline(x=1e-5, color='red', linestyle='--', label='Label')
        # Add Legend Handle
        handles, labels = a.get_legend_handles_labels()
        # Create the custom text as a Line2D object
        extra = Line2D([0], [0], color='none', label='')
        # Insert at the beginning of the list
        handles.insert(0, extra)
        labels.insert(0, legend_text[i])
        a.legend(handles=handles, labels=labels, loc='lower right')
    plt.savefig('Figur3')
    plt.show()

figureIII()

# === FIGURE IV: TPR Ratio vs. Bit Width for AE Models ===
# Data:
# - x-axis → Bit width (2, 4, 6, 8, 10, 12, 14, 16)
# - y-axis → TPR ratio (TPR for quantized model / TPR for baseline model)
# - Models:
#   - CNN AE (top left)
#   - DNN AE (top right)
#   - CNN AE (bottom left, QAT)
#   - DNN AE (bottom right, QAT)
# Method:
# - Use MSE-based anomaly score for AE models
# - Compute TPR ratio at FPR = 10⁻⁵

# === FIGURE V: TPR Ratio vs. Bit Width for VAE Models ===
# Data:
# - x-axis → Bit width (2, 4, 6, 8, 10, 12, 14, 16)
# - y-axis → TPR ratio (TPR for quantized model / TPR for baseline model)
# - Models:
#   - CNN VAE (top left)
#   - DNN VAE (top right)
#   - CNN VAE (bottom left, QAT)
#   - DNN VAE (bottom right, QAT)
# Method:
# - Use KL-based anomaly score for VAE models
# - Compute TPR ratio at FPR = 10⁻⁵

# === TABLE I: Performance Table at Floating-Point Precision ===
# Data:
# - Models:
#   - CNN AE, CNN VAE
#   - DNN AE, DNN VAE
# - AD Scores:
#   - IO (MSE)
#   - KL divergence
#   - Rz score
# - Benchmark Models:
#   - LQ → bτ
#   - A → 4ℓ
#   - h± → τν
#   - h0 → ττ
# - Metrics:
#   - AUC
#   - TPR @ FPR = 10⁻⁵

# === TABLE II: Performance of Quantized and Pruned Models ===
# Data:
# - Models:
#   - CNN AE (QAT 4-bit)
#   - CNN VAE (PTQ 8-bit)
#   - DNN AE (QAT 8-bit)
#   - DNN VAE (PTQ 8-bit)
# - AD Scores:
#   - IO (MSE)
#   - KL divergence
#   - Rz score
# - Metrics:
#   - AUC
#   - TPR @ FPR = 10⁻⁵

# === TABLE III: FPGA Resource Usage and Latency ===
# Data:
# - Models:
#   - CNN AE (QAT 4-bit)
#   - CNN VAE (PTQ 8-bit)
#   - DNN AE (QAT 8-bit)
#   - DNN VAE (PTQ 8-bit)
# - Metrics:
#   - DSP usage (%)
#   - LUT usage (%)
#   - Flip-Flop usage (%)
#   - BRAM usage (%)
#   - Latency (ns)
#   - Initiation interval (ns)

# === TABLE IV: FPGA Resource Usage for DNN AE ===
# Data:
# - Model:
#   - DNN VAE (PTQ 8-bit)
# - Metrics:
#   - DSP usage (%)
#   - LUT usage (%)
#   - Flip-Flop usage (%)
#   - BRAM usage (%)
#   - Latency (ns)
#   - Initiation interval (ns)
# - Results are for a Xilinx V7-690 FPGA



In [ ]:
dset = h5py.File('./content/leptoquark_LOWMASS_lepFilter_13TeV_filtered.h5', 'r')
dset = dset['Particles'][:,:,:3]
labels = np.ones(dset.shape[0])
#dset[:,:,0] = ((dset[:,:,0] - mean_pt) / std_pt)

#dnn_ae = tf.keras.models.load_model('content/full_dnn_ae.keras')

pred_sig, b = dn_vae.encoder.predict(dset)
pred_back, b = dn_vae.encoder.predict(data[:, :3])

from scipy.spatial import ConvexHull
def plot_hull(ax, points, color):
    hull = ConvexHull(points)
    for simplex in hull.simplices:
        ax.plot(points[simplex, 0], points[simplex, 1], points[simplex, 2], color=color)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(pred_sig[:,0], pred_sig[:,1], pred_sig[:,2], c='r', marker='o', label='Signal')
#ax.scatter(pred_back[:,0], pred_back[:,1], pred_back[:,2], c='b', marker='o', label='Background')

plot_hull(ax, pred_sig, 'red')
plot_hull(ax, pred_back, 'blue')

plt.legend()
plt.savefig('figur4')
plt.show()



In [ ]:
print(done)